In [1]:
import pandas as pd
import numpy as np

For the Decision Tree Regressor which only takes numerical input, the categorical variables must be one-hot-encoded.

In [2]:
acc_data = pd.read_csv('accordions.csv')
acc_data = acc_data[acc_data['price'] != 0]
acc_data = acc_data.drop(['accID', 'extra'], axis=1)

revamped_data = acc_data
revamped_data = revamped_data.drop('name', axis=1)

# One-hot-encoding brands
brand_vars = acc_data['brand'].unique()
for v in brand_vars:
    revamped_data.insert(2, v, (revamped_data['brand'] == v).astype('int32'))
revamped_data = revamped_data.drop('brand', axis=1)

# One-hot-encoding type
revamped_data.insert(8, 'Button', (revamped_data['type'] == 'Button').astype('int32'))
revamped_data.insert(8, 'Piano', (revamped_data['type'] == 'Piano').astype('int32'))
revamped_data = revamped_data.drop('type', axis=1)

# One-hot-encoding color (and generalizing)
colors = ['White', 'Black', 'Blue', 'Silver', 'Red', 'Green', 'Gray', 'Orange']
for c in colors:
    revamped_data.insert(7, c, (revamped_data['color'].str.contains(c)).astype('int32'))
revamped_data = revamped_data.drop('color', axis=1)
revamped_data


,price,preowned,Princetti,Hohnica,Roland,Hohner,Orange,Gray,Green,Red,Silver,Blue,Black,White,Piano,Button
0,8640.51,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
1,8447.00,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,7865.71,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,7865.71,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
4,7865.71,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
5,7865.71,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
6,7865.71,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
7,7865.71,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
8,7627.00,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
9,6555.00,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


Decision Tree Regressor

In [3]:
from sklearn.tree import DecisionTreeRegressor
features = revamped_data.drop('price', axis=1).columns

train_data = revamped_data.iloc[0:20, :]
validation_data = revamped_data.iloc[20:, :]

train_y = train_data['price']
train_X = train_data[features]

valid_y = validation_data['price']
valid_X = validation_data[features]

acc_model = DecisionTreeRegressor()
acc_model.fit(train_X, train_y)

DecisionTreeRegressor()

In [4]:
'''
Mean Absolute Error calculator

Subtracting the predicted price from the data for each, taking the absolute value of each, then averaging the total error.
'''
def MAE(data, predicted):
    return np.mean(abs(data - predicted))
    

predictions = acc_model.predict(valid_X)
MAE(valid_y, predictions)

4041.6511428571425

Custom accordion prediction

In [5]:
myaccordion = valid_X.drop(valid_X[valid_X['Button'] == 1].index)
myaccordion = myaccordion.drop(myaccordion[myaccordion['Button'] == 0].index) # Empty

# This accordion will be:
#    Hohner
#    Red
#    Piano
#    Preowned
custom_acc = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0])
myaccordion.loc[len(myaccordion)] = custom_acc

myacc_predict = acc_model.predict(myaccordion)
print(f'This Accordion will cost around ${myacc_predict[0]}')

This Accordion will cost around $5733.0


### Issues
The Mean Absolute Error is very poor meaning this is basically useless at actually guessing prices.

There simply is not enough data. The model is likely underfitted and there are not enough categorical variables to be accurate to actual accordion prices.